# Interactive Layer Clipping

## Step 1: Define your AOI

Draw a polygon on the map using the polygon control.  When you add a polygon, the bounding box
that the tool uses will be shown as a rectangle.

## Step 2: Define your target SRS

Provide your target SRS as an EPSG code.  The identified name of the EPSG code is provided for your reference.

If the EPSG code provided is for a geographic coordinate system, you will also be prompted for the cell size.

## Step 2: Click the button to start clipping

Logging will be printed below the map.

In [57]:
# How to get layer names passed in as URL parameters.
# 
# A button in the catalog can be programmatically built in this way
# to link to this jupyter notebook wherever it is served and have
# the notebook constructed so that we pre-select the target layer.
#
# https://stackoverflow.com/a/37134476

from IPython.display import HTML
HTML('''
    <script type="text/javascript">
        IPython.notebook.kernel.execute("URL = '" + window.location + "'")
    </script>''')


In [47]:
#print(URL)
import os
from urllib.parse import parse_qs

# localhost:8866?foo=bar,baz will return {'foo': ['bar', 'baz']}
query_string = os.environ.get('QUERY_STRING', '')
parameters = parse_qs(query_string)
display("query string parameters:", parameters)

'query string parameters:'

{}

In [48]:
import json
import threading
import logging

LOGGER = logging.getLogger(__name__)

import pygeoprocessing
import shapely.geometry
from osgeo import gdal, osr
import ipyleaflet
from ipyleaflet import Map, LocalTileLayer, basemap_to_tiles, basemaps
from ipyleaflet import LayersControl, ZoomControl, ScaleControl, LegendControl, WidgetControl, DrawControl
import IPython.display
import ipywidgets as widgets

gdal.DontUseExceptions()


# From https://ipywidgets.readthedocs.io/en/latest/examples/Output%20Widget.html
class OutputWidgetHandler(logging.Handler):
    """ Custom logging handler sending logs to an output widget """

    def __init__(self, *args, **kwargs):
        super(OutputWidgetHandler, self).__init__(*args, **kwargs)
        layout = {
            'width': '100%', 
            'height': '160px', 
            'border': '1px solid black'
        }
        self.out = widgets.Output(layout=layout)

    def emit(self, record):
        """ Overload of logging.Handler method """
        formatted_record = self.format(record)
        new_output = {
            'name': 'stdout', 
            'output_type': 'stream', 
            'text': formatted_record+'\n'
        }
        self.out.outputs = (new_output, ) + self.out.outputs
        
    def show_logs(self):
        """ Show the logs """
        display(self.out)
    
    def clear_logs(self):
        """ Clear the current logs """
        self.out.clear_output()
        
output_handler = OutputWidgetHandler()
output_handler.setFormatter(logging.Formatter('%(asctime)s  - [%(levelname)s] %(message)s'))
logging.getLogger().addHandler(output_handler)
logging.getLogger().setLevel(logging.INFO)
logging.getLogger('pygeoprocessing').setLevel(logging.DEBUG)


with open('stats.json') as stats_json:
    stats = json.load(stats_json)
    
minimum = stats['minimum']
maximum = stats['maximum']


chosen_basemap = basemap_to_tiles(basemaps.OpenStreetMap.Mapnik)
chosen_basemap.name = '(basemap) OpenStreetMap Mapnik'

m = Map(
    basemap=chosen_basemap,
    center=(0,0),
    zoom=2,  # zoom level for most of the globedisplay(m)
)  

scalebar = ScaleControl(position='bottomleft')
m.add_control(scalebar)

# If we want to pre-cache these tiles on GCS, use ipyleaflet.leaflet.TileLayer instead
country_id_tiles_layer = LocalTileLayer(
    path='tiles/{z}/{x}/{y}.png',
    name='NASA SRTM',
    show_loading=True,
    attribution='NASA'
    
)
m.add_layer(country_id_tiles_layer)

layerlist = LayersControl(position='topleft')
m.add_control(layerlist)

# build a colormap from our GDAL-compatible GRASS color file
import branca
colors = []
with open('elevation-color-profile-grass.txt') as color_profile:
    for line in color_profile:
        try:
            percent, r, g, b = [c.strip() for c in line.split(' ') if c]
        except ValueError:
            pass
        colors.append((float(r), float(g), float(b)))

colormap = branca.colormap.LinearColormap(
    colors=colors,
    vmin=minimum, vmax=maximum)
colormap.caption = 'Elevation (m)'
html = f"{colormap._repr_html_()}"

colorbar = WidgetControl(widget=widgets.HTML(html), position='topright')
m.add_control(colorbar)


draw_control = DrawControl(polyline={}, circlemarker={})
draw_control.polygon = {
    "shapeOptions": {
        "fillColor": "#6be5c3",
        "color": "#6be5c3",
        "fillOpacity": 0.5
    },
    "drawError": {
        "color": "#dd253b",
        "message": "Oups!"
    },
    "allowIntersection": False
}
m.add_control(draw_control)

submit_button = widgets.Button(
    description = 'Clip layer to AOI',
    disabled=False,  # start out disabled, enable when geometries defined
    button_style='',  # change to 'success' when geometries added
    tooltip='Clip the underlying layer by the bounding box of the geometries defined.',
    icon='scissors',  # this is a fontawesome icon
)
submit_button_control = WidgetControl(widget=submit_button, position='bottomright')


def _add_submit_button(control, **kwargs):
    # enable the submit button if 
    if kwargs['geo_json']['geometry']['coordinates']:
        m.add_control(submit_button_control)
    else:
        try:
            m.remove_control(submit_button_control)
        except ipyleaflet.ControlException:
            # When the control is not currently on the map
            pass
        
draw_control.on_draw(_add_submit_button)

WGS84_SRS = osr.SpatialReference()
WGS84_SRS.ImportFromEPSG(4326)


def compute(source_raster_path, aoi_geom, target_epsg, target_raster_path, target_pixel_size):
    LOGGER.info('bar')
    LOGGER.info('Warping')
    # Clip the source raster to the target bounding box in WGS84
    raster_info = pygeoprocessing.get_raster_info(source_raster_path)
    target_srs = osr.SpatialReference()
    target_srs.ImportFromEPSG(target_epsg)
    target_srs_wkt = target_srs.ExportToWkt()
    target_bbox = pygeoprocessing.transform_bounding_box(
        bounding_box=shapely.geometry.shape(aoi_geom).bounds,
        base_projection_wkt=WGS84_SRS.ExportToWkt(),
        target_projection_wkt=target_srs_wkt,
    )
    
    pygeoprocessing.warp_raster(
        base_raster_path=source_raster_path,
        target_pixel_size=target_pixel_size,
        target_raster_path=target_raster_path,
        resample_method='near',
        target_projection_wkt=target_srs_wkt,
        target_bb=target_bbox,
    )
    LOGGER.info('Finished warping')
    LOGGER.info(f"Clipped raster available at {target_raster_path}")
    print('ending')


def _submit_form(*args, **kwargs):
    LOGGER.info('foo')
    submit_button.description = "Clipping ..."
    submit_button.disabled = True
    args=[
            'intermediate/global_dem.tif',
            draw_control.data[0]['geometry'],
            int(epsg_input.value),
            'foo.tif',
            (float(pixel_size_x_input.value), float(pixel_size_y_input.value)),
        ]
    compute(*args)
    
    submit_button.disabled = False
    submit_button.description = 'Clip layer to AOI'
    print('bar')
    
submit_button.on_click(_submit_form)


epsg_input = widgets.Text(
    value='4326',
    placeholder='4326',
    description='EPSG code:',
    disabled=False
)
epsg_label = widgets.Label(value="")
pixel_sizes_template = "NOTE: pixel sizes have units: {units}"
pixel_sizes_label = widgets.HTML("")
def _detect_srs_name(change):
    try:
        if isinstance(change['new'], str):
            epsg_code = int(change['new'])
        elif isinstance(change['new'], dict):
            epsg_code = int(change['new']['value'])
        else:
            raise Exception(change)
    except (ValueError, TypeError):
        epsg_code = None
    except KeyError:
        # no new value was set; skip
        return
    
    if isinstance(epsg_code, int):
        # TODO: silence warnings from bad EPSG codes
        srs = osr.SpatialReference()
        srs.ImportFromEPSG(epsg_code)
        srs_name = srs.GetName()
        if srs.IsGeographic():
            srs_units = srs.GetAngularUnitsName()
        else:
            srs_units = srs.GetLinearUnitsName()

        if srs_name in {None, 'unknown'}:
            label = f"EPSG code {epsg_code} not recognized"
            pixel_size_text = ""
        else:
            label = f"{srs_name}, Units: {srs_units}"
            pixel_size_text = pixel_sizes_template.format(
                units=srs_units)
    else:
        label = f"EPSG code {epsg_code} not recognized"
        pixel_size_text = ""
        
    epsg_label.value = label
    pixel_sizes_label.value = pixel_size_text
    
epsg_input.observe(_detect_srs_name)

pixel_size_x_input = widgets.Text(
    value='1',
    placeholder='1',
    description='Pixel X size',
    disabled=False
)
pixel_size_y_input = widgets.Text(
    value='1',
    placeholder='1',
    description='Pixel Y size',
    disabled=False
)

output_params_box = widgets.VBox([
    widgets.HTML("<b>Output options</b>"),
    widgets.HBox([epsg_input, epsg_label]),
    widgets.HBox(
        [pixel_size_x_input, pixel_size_y_input, pixel_sizes_label]),
])

logging_box = widgets.VBox([
    widgets.HTML("<b>Logging</b>"),
    output_handler.out,
])

display(m)
display(output_params_box)
display(logging_box)

Map(center=[0, 0], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_text'…

ERROR 1: PROJ: proj_create_from_database: crs not found
ERROR 1: PROJ: proj_create_from_database: crs not found
ERROR 1: PROJ: proj_create_from_database: crs not found
ERROR 1: PROJ: proj_create_from_database: crs not found
ERROR 1: PROJ: proj_create_from_database: crs not found
ERROR 1: PROJ: proj_create_from_database: crs not found
INFO:__main__:foo
INFO:__main__:bar
INFO:__main__:Warping
INFO:__main__:Finished warping
INFO:__main__:Clipped raster available at foo.tif


ending
bar


Output(layout=Layout(border_bottom='1px solid black', border_left='1px solid black', border_right='1px solid b…

http://localhost:8889/notebooks/demo.ipynb


In [49]:
# Remaining TODOs for this application:
# - [x] get the URL when running in Voila mode
# - [ ] expand this to operate on various layers available
# - [x] Allow the user to define their target EPSG code
# - [x] Use the user's target EPSG code in warping
# - [x] If the EPSG code represents a projected coord system (!osr.IsGeographic()), provide the pixel size
# - [x] Use osr.SpatialReference().GetName() to display the name of the coordinate system for user reference (unknown if unknown)